<a href="https://colab.research.google.com/github/Englishson0909/2024spring/blob/main/textextractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yt-dlp speechrecognition pydub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.6 MB/s eta 0:00:00


In [3]:
python youtube_to_text.py


SyntaxError: invalid syntax (<ipython-input-3-93a579da5318>, line 1)

In [4]:
# Install necessary packages if you haven't already
# You can do this by running:
# pip install yt-dlp speechrecognition pydub

import yt_dlp
import speech_recognition as sr
from pydub import AudioSegment
import os

# Function to download the YouTube video and extract audio
def download_youtube_audio(youtube_url, output_audio_path='audio.wav'):
    print("Downloading audio from YouTube...")
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'audio.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

    # Convert the mp3 file to wav format
    print("Converting audio to WAV format...")
    audio = AudioSegment.from_file('audio.mp3', format="mp3")
    audio.export(output_audio_path, format="wav")

    # Remove the temporary mp3 file
    os.remove('audio.mp3')

    return output_audio_path

# Function to transcribe audio to text
def transcribe_audio(audio_path):
    print("Transcribing audio to text...")
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        return "Google Speech Recognition could not understand the audio"
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service; {e}"

if __name__ == "__main__":
    youtube_url = input("Enter YouTube video URL: ")
    audio_path = download_youtube_audio(youtube_url)
    transcript = transcribe_audio(audio_path)
    print("Transcript:\n", transcript)


Enter YouTube video URL: https://youtu.be/GOlCXVWoNVc?t=6
[youtube] Extracting URL: https://youtu.be/GOlCXVWoNVc?t=6
[youtube] GOlCXVWoNVc: Downloading webpage
[youtube] GOlCXVWoNVc: Downloading ios player API JSON
[youtube] GOlCXVWoNVc: Downloading web creator player API JSON
[youtube] GOlCXVWoNVc: Downloading player 6db2bd17
[youtube] GOlCXVWoNVc: Downloading m3u8 information
[info] GOlCXVWoNVc: Downloading 1 format(s): 251
[download] Destination: audio.webm
[download] 100% of    4.37MiB in 00:00:00 at 10.37MiB/s  
[ExtractAudio] Destination: audio.mp3
Deleting original file audio.webm (pass -k to keep)
Converting audio to WAV format...
Transcribing audio to text...
Transcript:
 Could not request results from Google Speech Recognition service; recognition connection failed: [Errno 32] Broken pipe


In [5]:
def transcribe_audio_in_chunks(audio_path, chunk_duration=30):
    print("Transcribing audio to text in chunks...")
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_wav(audio_path)
    duration = len(audio) // 1000  # Convert milliseconds to seconds
    transcript = []

    for i in range(0, duration, chunk_duration):
        chunk = audio[i*1000:(i+chunk_duration)*1000]
        chunk.export("chunk.wav", format="wav")
        with sr.AudioFile("chunk.wav") as source:
            audio_chunk = recognizer.record(source)
            try:
                text = recognizer.recognize_google(audio_chunk)
                transcript.append(text)
            except sr.UnknownValueError:
                transcript.append("[Unintelligible]")
            except sr.RequestError as e:
                transcript.append(f"[Error: {e}]")

    os.remove("chunk.wav")
    return " ".join(transcript)


In [6]:
if __name__ == "__main__":
    youtube_url = input("Enter YouTube video URL: ")
    audio_path = download_youtube_audio(youtube_url)
    transcript = transcribe_audio_in_chunks(audio_path)
    print("Transcript:\n", transcript)


Enter YouTube video URL: https://youtu.be/GOlCXVWoNVc
[youtube] Extracting URL: https://youtu.be/GOlCXVWoNVc
[youtube] GOlCXVWoNVc: Downloading webpage
[youtube] GOlCXVWoNVc: Downloading ios player API JSON
[youtube] GOlCXVWoNVc: Downloading web creator player API JSON
[youtube] GOlCXVWoNVc: Downloading m3u8 information
[info] GOlCXVWoNVc: Downloading 1 format(s): 251
[download] Destination: audio.webm
[download] 100% of    4.37MiB in 00:00:00 at 21.45MiB/s  
[ExtractAudio] Destination: audio.mp3
Deleting original file audio.webm (pass -k to keep)
Converting audio to WAV format...
Transcribing audio to text in chunks...
Transcript:
 sometimes I think I'm cursed sometimes I think I'm cursed sometimes I think I'm cursed sometimes I think I'm cursed sometimes I think I'm cursed because of something that happened before I was even born because of something that happened before I was even born because of something that happened before I was even born because of something that happened befor